In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
from tensorflow.python.client import device_lib

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print(gpu)

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [2]:
#모든 열 보기
pd.set_option('display.max_columns', None)

keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [3]:
path = 'D:/Dataset/Competitons/oil_construction_machine/'
train = pd.read_csv(path + 'train.csv').copy()
test = pd.read_csv(path + 'test.csv').copy()
origin_train = train.copy()
train

,ID,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,SAMPLE_TRANSFER_DAY,ANONYMOUS_2,AG,AL,B,BA,BE,CA,CD,CO,CR,CU,FH2O,FNOX,FOPTIMETHGLY,FOXID,FSO4,FTBN,FE,FUEL,H2O,K,LI,MG,MN,MO,NA,NI,P,PB,PQINDEX,S,SB,SI,SN,SOOTPERCENTAGE,TI,U100,U75,U50,U25,U20,U14,U6,U4,V,V100,V40,ZN,Y_LABEL
0,TRAIN_00000,COMPONENT3,1486,2011,7,200,0,3,93,0,0,3059,0.0,0,13,78,NaN,NaN,NaN,NaN,NaN,NaN,888,NaN,0.0,27.0,1,14,16,1,16,6,1951,0,8504,21370,0,427,2,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,154.0,75,0
1,TRAIN_00001,COMPONENT2,1350,2021,51,375,0,2,19,0,0,2978,0.0,0,0,31,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,0.0,NaN,0,1,0,0,1,0,572,2,19,1117,0,0,0,NaN,0,0.0,0.0,1.0,2.0,4.0,6.0,216.0,1454.0,0,NaN,44.0,652,0
2,TRAIN_00002,COMPONENT2,2415,2015,2,200,0,110,1,1,0,17,0.0,0,1,2,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,0.0,0.0,0,0,0,0,2,0,328,0,17,1334,0,0,1,NaN,0,0.0,0.0,0.0,0.0,3.0,39.0,11261.0,41081.0,0,NaN,72.6,412,1
3,TRAIN_00003,COMPONENT3,7389,2010,2,200,0,8,3,0,0,1960,0.0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,37,NaN,0.0,NaN,0,4,1,0,0,0,906,1,44,21774,2,1,1,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,133.3,7,0
4,TRAIN_00004,COMPONENT3,3954,2015,4,200,0,1,157,0,0,71,0.0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,71,NaN,0.0,0.0,0,0,0,0,2,0,309,0,217,18470,0,2,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,133.1,128,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14090,TRAIN_14090,COMPONENT3,1616,2014,8,200,0,2,201,1,0,6,0.0,0,0,3,NaN,NaN,NaN,NaN,NaN,NaN,23,NaN,0.0,1.0,0,0,0,0,8,0,477,0,35,22000,0,2,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,135.4,16,0
14091,TRAIN_14091,COMPONENT1,2784,2013,2,200,0,3,85,0,0,2945,0.0,0,0,2,13.0,8.0,0.0,16.0,21.0,9.1,12,0.0,0.0,4.0,0,36,0,224,3,0,1163,1,9,5494,0,5,0,0.4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,14.5,117.5,1408,0
14092,TRAIN_14092,COMPONENT3,1788,2008,9,550,0,6,0,1,0,13,0.0,0,4,7,NaN,NaN,NaN,NaN,NaN,NaN,415,NaN,0.0,NaN,1,16,7,10,3,1,1175,1,645,9640,2,18,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,54.0,1301,0
14093,TRAIN_14093,COMPONENT2,2498,2009,19,550,0,2,4,0,0,2244,NaN,0,0,170,NaN,NaN,NaN,NaN,NaN,NaN,19,NaN,0.0,0.0,0,0,0,1,0,0,441,1,11,1313,0,0,0,NaN,0,0.0,0.0,0.0,7.0,8.0,100.0,1625.0,18890.0,0,NaN,44.3,652,0


In [4]:
categorical_features = ['COMPONENT_ARBITRARY', 'YEAR']
# Inference(실제 진단 환경)에 사용하는 컬럼
test_stage_features = ['COMPONENT_ARBITRARY', 'ANONYMOUS_1', 'YEAR' , 'ANONYMOUS_2', 'AG', 'CO', 'CR', 'CU', 'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V', 'V40', 'ZN']

In [5]:
train.isna().sum()

ID                         0
COMPONENT_ARBITRARY        0
ANONYMOUS_1                0
YEAR                       0
SAMPLE_TRANSFER_DAY        0
ANONYMOUS_2                0
AG                         0
AL                         0
B                          0
BA                         0
BE                         0
CA                         0
CD                      1394
CO                         0
CR                         0
CU                         0
FH2O                   10205
FNOX                   10205
FOPTIMETHGLY           10205
FOXID                  10205
FSO4                   10205
FTBN                   10205
FE                         0
FUEL                   10205
H2O                        0
K                       2299
LI                         0
MG                         0
MN                         0
MO                         0
NA                         0
NI                         0
P                          0
PB                         0
PQINDEX       

In [6]:
data_info = pd.read_excel(path + 'data_info.xlsx')
data_info

,Feature Name,Description,단위,비고
0,COMPONENT_ARBITRARY,"샘플 오일 관련 부품 (Component 4종, 비식별화)",NaN,NaN
1,ANONYMOUS_1,무명 Feautre 1. 수치형 데이터,NaN,NaN
2,YEAR,오일 샘플 및 진단 해 (Year),NaN,NaN
3,SAMPLE_TRANSFER_DAY,오일 샘플링 후 진단 기관으로 이동한 기간 (Days),NaN,NaN
4,ANONYMOUS_2,무명 Feautre 2. 수치형 데이터,NaN,NaN
5,AG,원소기호 AG - Silver 함유량,ppm,NaN
6,AL,원소기호 AL - Aluminium 함유량,ppm,NaN
7,B,원소기호 B - Boron 함유량,ppm,NaN
8,BA,원소기호 BA - Barium 함유량,ppm,NaN
9,BE,원소기호 BE - Beryllium 함유량,ppm,NaN


In [7]:
test

,ID,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,ANONYMOUS_2,AG,CO,CR,CU,FE,H2O,MN,MO,NI,PQINDEX,TI,V,V40,ZN
0,TEST_0000,COMPONENT1,2192,2016,200,0,0,0,1,12,0.0,0,0,0,10,0,0,91.3,1091
1,TEST_0001,COMPONENT3,2794,2011,200,0,0,2,1,278,0.0,3,0,0,2732,1,0,126.9,12
2,TEST_0002,COMPONENT2,1982,2010,200,0,0,0,16,5,0.0,0,0,0,11,0,0,44.3,714
3,TEST_0003,COMPONENT3,1404,2009,200,0,0,3,4,163,0.0,4,3,0,8007,0,0,142.8,94
4,TEST_0004,COMPONENT2,8225,2013,200,0,0,0,6,13,0.0,0,0,0,16,0,0,63.4,469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,TEST_6036,COMPONENT3,1714,2014,200,0,0,3,130,1047,0.0,65,5,2,1999,0,0,71.2,1163
6037,TEST_6037,COMPONENT3,4131,2016,200,0,0,5,2,736,0.0,5,0,0,693,0,0,137.8,13
6038,TEST_6038,COMPONENT3,4325,2014,200,0,0,0,0,53,0.0,0,0,0,69,0,0,302.2,13
6039,TEST_6039,COMPONENT2,1364,2013,200,0,0,0,62,2,0.0,0,0,0,5,0,0,51.1,478


In [8]:
len(test.columns.unique())

19

In [9]:
len(train.columns.unique())

54

In [10]:
train = train.fillna(0)
test = test.fillna(0)
train.isna().sum()

ID                     0
COMPONENT_ARBITRARY    0
ANONYMOUS_1            0
YEAR                   0
SAMPLE_TRANSFER_DAY    0
ANONYMOUS_2            0
AG                     0
AL                     0
B                      0
BA                     0
BE                     0
CA                     0
CD                     0
CO                     0
CR                     0
CU                     0
FH2O                   0
FNOX                   0
FOPTIMETHGLY           0
FOXID                  0
FSO4                   0
FTBN                   0
FE                     0
FUEL                   0
H2O                    0
K                      0
LI                     0
MG                     0
MN                     0
MO                     0
NA                     0
NI                     0
P                      0
PB                     0
PQINDEX                0
S                      0
SB                     0
SI                     0
SN                     0
SOOTPERCENTAGE         0


In [11]:
test.isna().sum()

ID                     0
COMPONENT_ARBITRARY    0
ANONYMOUS_1            0
YEAR                   0
ANONYMOUS_2            0
AG                     0
CO                     0
CR                     0
CU                     0
FE                     0
H2O                    0
MN                     0
MO                     0
NI                     0
PQINDEX                0
TI                     0
V                      0
V40                    0
ZN                     0
dtype: int64

In [12]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
LE = LabelEncoder()

train['COMPONENT_ARBITRARY'] = LE.fit_transform(train[['COMPONENT_ARBITRARY']])
test['COMPONENT_ARBITRARY'] = LE.transform(test[['COMPONENT_ARBITRARY']])

onehot_features = ['COMPONENT_ARBITRARY']

for i in onehot_features:
    onehot = OneHotEncoder(sparse = False)
    train_onehot = onehot.fit_transform(train[[i]])
    train_onehot = pd.DataFrame(train_onehot, columns = [i + '_' + str(col) for col in onehot.categories_[0]])
    train = pd.concat([train.drop(columns = [i]), train_onehot], axis = 1)
    
    test_onehot = onehot.transform(test[[i]])
    test_onehot = pd.DataFrame(test_onehot, columns = [i + '_' + str(col) for col in onehot.categories_[0]])
    test = pd.concat([test.drop(columns = [i]), test_onehot], axis = 1)

train

C:\Users\cym80\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\cym80\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,ID,ANONYMOUS_1,YEAR,SAMPLE_TRANSFER_DAY,ANONYMOUS_2,AG,AL,B,BA,BE,CA,CD,CO,CR,CU,FH2O,FNOX,FOPTIMETHGLY,FOXID,FSO4,FTBN,FE,FUEL,H2O,K,LI,MG,MN,MO,NA,NI,P,PB,PQINDEX,S,SB,SI,SN,SOOTPERCENTAGE,TI,U100,U75,U50,U25,U20,U14,U6,U4,V,V100,V40,ZN,Y_LABEL,COMPONENT_ARBITRARY_0,COMPONENT_ARBITRARY_1,COMPONENT_ARBITRARY_2,COMPONENT_ARBITRARY_3
0,TRAIN_00000,1486,2011,7,200,0,3,93,0,0,3059,0.0,0,13,78,0.0,0.0,0.0,0.0,0.0,0.0,888,0.0,0.0,27.0,1,14,16,1,16,6,1951,0,8504,21370,0,427,2,0.0,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,154.0,75,0,0.0,0.0,1.0,0.0
1,TRAIN_00001,1350,2021,51,375,0,2,19,0,0,2978,0.0,0,0,31,0.0,0.0,0.0,0.0,0.0,0.0,2,0.0,0.0,0.0,0,1,0,0,1,0,572,2,19,1117,0,0,0,0.0,0,0.0,0.0,1.0,2.0,4.0,6.0,216.0,1454.0,0,0.0,44.0,652,0,0.0,1.0,0.0,0.0
2,TRAIN_00002,2415,2015,2,200,0,110,1,1,0,17,0.0,0,1,2,0.0,0.0,0.0,0.0,0.0,0.0,4,0.0,0.0,0.0,0,0,0,0,2,0,328,0,17,1334,0,0,1,0.0,0,0.0,0.0,0.0,0.0,3.0,39.0,11261.0,41081.0,0,0.0,72.6,412,1,0.0,1.0,0.0,0.0
3,TRAIN_00003,7389,2010,2,200,0,8,3,0,0,1960,0.0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,37,0.0,0.0,0.0,0,4,1,0,0,0,906,1,44,21774,2,1,1,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,133.3,7,0,0.0,0.0,1.0,0.0
4,TRAIN_00004,3954,2015,4,200,0,1,157,0,0,71,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,71,0.0,0.0,0.0,0,0,0,0,2,0,309,0,217,18470,0,2,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,133.1,128,0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14090,TRAIN_14090,1616,2014,8,200,0,2,201,1,0,6,0.0,0,0,3,0.0,0.0,0.0,0.0,0.0,0.0,23,0.0,0.0,1.0,0,0,0,0,8,0,477,0,35,22000,0,2,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,135.4,16,0,0.0,0.0,1.0,0.0
14091,TRAIN_14091,2784,2013,2,200,0,3,85,0,0,2945,0.0,0,0,2,13.0,8.0,0.0,16.0,21.0,9.1,12,0.0,0.0,4.0,0,36,0,224,3,0,1163,1,9,5494,0,5,0,0.4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,14.5,117.5,1408,0,1.0,0.0,0.0,0.0
14092,TRAIN_14092,1788,2008,9,550,0,6,0,1,0,13,0.0,0,4,7,0.0,0.0,0.0,0.0,0.0,0.0,415,0.0,0.0,0.0,1,16,7,10,3,1,1175,1,645,9640,2,18,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,54.0,1301,0,0.0,0.0,1.0,0.0
14093,TRAIN_14093,2498,2009,19,550,0,2,4,0,0,2244,0.0,0,0,170,0.0,0.0,0.0,0.0,0.0,0.0,19,0.0,0.0,0.0,0,0,0,1,0,0,441,1,11,1313,0,0,0,0.0,0,0.0,0.0,0.0,7.0,8.0,100.0,1625.0,18890.0,0,0.0,44.3,652,0,0.0,1.0,0.0,0.0


In [13]:
train

,ID,ANONYMOUS_1,YEAR,SAMPLE_TRANSFER_DAY,ANONYMOUS_2,AG,AL,B,BA,BE,CA,CD,CO,CR,CU,FH2O,FNOX,FOPTIMETHGLY,FOXID,FSO4,FTBN,FE,FUEL,H2O,K,LI,MG,MN,MO,NA,NI,P,PB,PQINDEX,S,SB,SI,SN,SOOTPERCENTAGE,TI,U100,U75,U50,U25,U20,U14,U6,U4,V,V100,V40,ZN,Y_LABEL,COMPONENT_ARBITRARY_0,COMPONENT_ARBITRARY_1,COMPONENT_ARBITRARY_2,COMPONENT_ARBITRARY_3
0,TRAIN_00000,1486,2011,7,200,0,3,93,0,0,3059,0.0,0,13,78,0.0,0.0,0.0,0.0,0.0,0.0,888,0.0,0.0,27.0,1,14,16,1,16,6,1951,0,8504,21370,0,427,2,0.0,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,154.0,75,0,0.0,0.0,1.0,0.0
1,TRAIN_00001,1350,2021,51,375,0,2,19,0,0,2978,0.0,0,0,31,0.0,0.0,0.0,0.0,0.0,0.0,2,0.0,0.0,0.0,0,1,0,0,1,0,572,2,19,1117,0,0,0,0.0,0,0.0,0.0,1.0,2.0,4.0,6.0,216.0,1454.0,0,0.0,44.0,652,0,0.0,1.0,0.0,0.0
2,TRAIN_00002,2415,2015,2,200,0,110,1,1,0,17,0.0,0,1,2,0.0,0.0,0.0,0.0,0.0,0.0,4,0.0,0.0,0.0,0,0,0,0,2,0,328,0,17,1334,0,0,1,0.0,0,0.0,0.0,0.0,0.0,3.0,39.0,11261.0,41081.0,0,0.0,72.6,412,1,0.0,1.0,0.0,0.0
3,TRAIN_00003,7389,2010,2,200,0,8,3,0,0,1960,0.0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,37,0.0,0.0,0.0,0,4,1,0,0,0,906,1,44,21774,2,1,1,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,133.3,7,0,0.0,0.0,1.0,0.0
4,TRAIN_00004,3954,2015,4,200,0,1,157,0,0,71,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,71,0.0,0.0,0.0,0,0,0,0,2,0,309,0,217,18470,0,2,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,133.1,128,0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14090,TRAIN_14090,1616,2014,8,200,0,2,201,1,0,6,0.0,0,0,3,0.0,0.0,0.0,0.0,0.0,0.0,23,0.0,0.0,1.0,0,0,0,0,8,0,477,0,35,22000,0,2,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,135.4,16,0,0.0,0.0,1.0,0.0
14091,TRAIN_14091,2784,2013,2,200,0,3,85,0,0,2945,0.0,0,0,2,13.0,8.0,0.0,16.0,21.0,9.1,12,0.0,0.0,4.0,0,36,0,224,3,0,1163,1,9,5494,0,5,0,0.4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,14.5,117.5,1408,0,1.0,0.0,0.0,0.0
14092,TRAIN_14092,1788,2008,9,550,0,6,0,1,0,13,0.0,0,4,7,0.0,0.0,0.0,0.0,0.0,0.0,415,0.0,0.0,0.0,1,16,7,10,3,1,1175,1,645,9640,2,18,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,54.0,1301,0,0.0,0.0,1.0,0.0
14093,TRAIN_14093,2498,2009,19,550,0,2,4,0,0,2244,0.0,0,0,170,0.0,0.0,0.0,0.0,0.0,0.0,19,0.0,0.0,0.0,0,0,0,1,0,0,441,1,11,1313,0,0,0,0.0,0,0.0,0.0,0.0,7.0,8.0,100.0,1625.0,18890.0,0,0.0,44.3,652,0,0.0,1.0,0.0,0.0


In [14]:
test

,ID,ANONYMOUS_1,YEAR,ANONYMOUS_2,AG,CO,CR,CU,FE,H2O,MN,MO,NI,PQINDEX,TI,V,V40,ZN,COMPONENT_ARBITRARY_0,COMPONENT_ARBITRARY_1,COMPONENT_ARBITRARY_2,COMPONENT_ARBITRARY_3
0,TEST_0000,2192,2016,200,0,0,0,1,12,0.0,0,0,0,10,0,0,91.3,1091,1.0,0.0,0.0,0.0
1,TEST_0001,2794,2011,200,0,0,2,1,278,0.0,3,0,0,2732,1,0,126.9,12,0.0,0.0,1.0,0.0
2,TEST_0002,1982,2010,200,0,0,0,16,5,0.0,0,0,0,11,0,0,44.3,714,0.0,1.0,0.0,0.0
3,TEST_0003,1404,2009,200,0,0,3,4,163,0.0,4,3,0,8007,0,0,142.8,94,0.0,0.0,1.0,0.0
4,TEST_0004,8225,2013,200,0,0,0,6,13,0.0,0,0,0,16,0,0,63.4,469,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,TEST_6036,1714,2014,200,0,0,3,130,1047,0.0,65,5,2,1999,0,0,71.2,1163,0.0,0.0,1.0,0.0
6037,TEST_6037,4131,2016,200,0,0,5,2,736,0.0,5,0,0,693,0,0,137.8,13,0.0,0.0,1.0,0.0
6038,TEST_6038,4325,2014,200,0,0,0,0,53,0.0,0,0,0,69,0,0,302.2,13,0.0,0.0,1.0,0.0
6039,TEST_6039,1364,2013,200,0,0,0,62,2,0.0,0,0,0,5,0,0,51.1,478,0.0,1.0,0.0,0.0


In [15]:
from sklearn.model_selection import train_test_split
all_X = train.drop(['ID', 'Y_LABEL'], axis = 1)
all_y = train['Y_LABEL']
test_train = test.drop(['ID'], axis = 1)

In [16]:
# from sklearn.preprocessing import StandardScaler
# std = StandardScaler()

# all_X = std.fit_transform(all_X[test_train.columns.unique()])
# test_train = std.transform(test_train)

In [17]:
X_train, X_val, y_train, y_val = train_test_split(all_X, all_y, test_size=0.2, random_state=42)

In [18]:
# X_train.info()

In [19]:
test.columns.unique()

Index(['ID', 'ANONYMOUS_1', 'YEAR', 'ANONYMOUS_2', 'AG', 'CO', 'CR', 'CU',
       'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V', 'V40', 'ZN',
       'COMPONENT_ARBITRARY_0', 'COMPONENT_ARBITRARY_1',
       'COMPONENT_ARBITRARY_2', 'COMPONENT_ARBITRARY_3'],
      dtype='object')

In [20]:
from lightgbm import LGBMClassifier
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.svm import SVC

teacher_lgbm = LGBMClassifier(random_state = 42)
teacher_cat = CatBoostClassifier(random_state = 42, verbose = 0)
teacher_xgb = XGBClassifier(random_state = 42)
teacher_svc = SVC()

teacher_stacking = StackingClassifier(estimators = [('lgbm', teacher_lgbm),
                                                   ('cat', teacher_cat),
                                                   ('xgb', teacher_xgb),
                                                   ('svc', teacher_svc)], verbose = 1,
                                     final_estimator = RandomForestClassifier(random_state = 42))
teacher_stacking.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    1.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   22.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    3.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    9.7s finished


StackingClassifier(estimators=[('lgbm', LGBMClassifier(random_state=42)),
                               ('cat',
                                <catboost.core.CatBoostClassifier object at 0x0000027181344220>),
                               ('xgb',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric=None, gamma=No...
                                              learning_rate=None, max_bin=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=None,
                                              max_depth=None, max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              n_estimators=100, n_jobs=None,
                                              num_parallel_tree=None,
                                              predictor=None, random_state=42,
                                              reg_alpha=None, reg_lambda=None, ...)),
                               ('svc', SVC())],
                   final_estimator=RandomForestClassifier(random_state=42),
                   verbose=1)

In [21]:
from sklearn.metrics import f1_score

pred = teacher_stacking.predict(X_val)
f1_score(y_val, pred)
# pred

0.6285714285714286

In [22]:
teacher_stacking

StackingClassifier(estimators=[('lgbm', LGBMClassifier(random_state=42)),
                               ('cat',
                                <catboost.core.CatBoostClassifier object at 0x0000027181344220>),
                               ('xgb',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric=None, gamma=No...
                                              learning_rate=None, max_bin=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=None,
                                              max_depth=None, max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              n_estimators=100, n_jobs=None,
                                              num_parallel_tree=None,
                                              predictor=None, random_state=42,
                                              reg_alpha=None, reg_lambda=None, ...)),
                               ('svc', SVC())],
                   final_estimator=RandomForestClassifier(random_state=42),
                   verbose=1)

In [23]:
teacher_pred = teacher_stacking.predict_proba(X_train)
teacher_pred

array([[0.99, 0.01],
       [0.54, 0.46],
       [0.98, 0.02],
       ...,
       [0.98, 0.02],
       [0.98, 0.02],
       [0.97, 0.03]])

In [24]:
teacher_stacking.predict(X_train)

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [25]:
teacher_stacking.predict(X_val)

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [26]:
from sklearn.tree import DecisionTreeRegressor

student = DecisionTreeRegressor(random_state = 42)
student.fit(X_train[test_train.columns.unique()],teacher_pred)

DecisionTreeRegressor(random_state=42)

In [27]:
student_pred = student.predict(X_val[test_train.columns.unique()])

In [28]:
student_pred

array([[1.  , 0.  ],
       [0.98, 0.02],
       [1.  , 0.  ],
       ...,
       [1.  , 0.  ],
       [0.96, 0.04],
       [0.97, 0.03]])

In [29]:
pred = np.argmax(student_pred, axis = 1)
pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [30]:
from sklearn.metrics import f1_score

f1_score(y_val, pred)

0.12177985948477753

In [31]:
y_val

7013     0
903      0
6997     0
1586     0
455      1
        ..
8635     0
6329     0
4667     0
12760    0
2948     0
Name: Y_LABEL, Length: 2819, dtype: int64

In [32]:
pred = student.predict(test_train)
pred = np.argmax(pred, axis = 1)
pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [34]:
output = pd.DataFrame({'ID': test.ID, 'Y_LABEL': pred})
output.to_csv('onehot_stacking_tree.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
